In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from foodspec.core.spectral_dataset import HyperspectralDataset
from foodspec.features.rq import PeakDefinition, RatioDefinition, RatioQualityEngine, RQConfig

print("✓ Imports successful")

## Step 1: Create Synthetic Hyperspectral Cube

Generate a small synthetic cube for demonstration.

In [ ]:
print("Creating synthetic hyperspectral cube...\n")
y_pix, x_pix, n_wn = 5, 4, 3
wn = np.array([1000.0, 1100.0, 1200.0])
cube = np.random.rand(y_pix, x_pix, n_wn)
meta = pd.DataFrame({"y": np.repeat(np.arange(y_pix), x_pix), 
                     "x": np.tile(np.arange(x_pix), y_pix)})
hsi = HyperspectralDataset.from_cube(cube, wn, metadata=meta)

print(f"Cube shape: {y_pix} (rows) × {x_pix} (cols) × {n_wn} (wavenumbers)")
print(f"Total pixels: {y_pix * x_pix}")
print(f"Wavenumber range: {wn[0]:.0f} - {wn[-1]:.0f} cm⁻¹")

## Step 2: Segment Image

Use K-means to identify spatial regions (e.g., food vs. background).

In [ ]:
print("Segmenting with K-means (k=2)...\n")
labels = hsi.segment(method="kmeans", n_clusters=2)

for k in np.unique(labels):
    n_pixels = (labels == k).sum()
    print(f"Cluster {k}: {n_pixels} pixels ({n_pixels/(y_pix*x_pix)*100:.1f}%)")

# Visualize label map
fig, ax = plt.subplots(figsize=(6, 5))
label_map = labels.reshape(y_pix, x_pix)
im = ax.imshow(label_map, cmap="viridis", interpolation="nearest")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.set_title("Segmentation Map (K-means clusters)")
plt.colorbar(im, ax=ax, label="Cluster ID")
plt.tight_layout()
plt.show()

## Step 3: Extract ROI Spectra

For each cluster, extract the mean spectrum.

In [ ]:
print("Extracting ROI spectra...\n")
roi_spectra = []
for k in np.unique(labels):
    mask = (labels == k)
    roi_ds = hsi.roi_spectrum(mask)
    roi_spectra.append(roi_ds)
    print(f"ROI {k}: {mask.sum()} pixels aggregated")

# Visualize ROI spectra
fig, ax = plt.subplots(figsize=(9, 5))
for idx, roi_ds in enumerate(roi_spectra):
    spectrum = roi_ds.spectra.squeeze()
    ax.plot(roi_ds.wavenumbers, spectrum, label=f"ROI {idx}", linewidth=2, marker='o')
ax.set_xlabel("Wavenumber (cm⁻¹)")
ax.set_ylabel("Mean Intensity")
ax.set_title("Mean Spectra by ROI")
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Step 4: RQ Analysis on ROI Data

Run Ratio Quality engine on aggregated ROI spectra.

In [ ]:
print("Running RQ analysis on ROI spectra...\n")

# Define peaks
peaks = [PeakDefinition(name=f"I_{int(w)}", column=f"I_{int(w)}", wavenumber=float(w)) for w in wn]
ratios = [RatioDefinition(name=f"I_{int(wn[0])}/I_{int(wn[1])}", 
                          numerator=f"I_{int(wn[0])}", denominator=f"I_{int(wn[1])}")
          for _ in range(1)]

# Create peak table from ROI spectra
dfs = []
for idx, roi_ds in enumerate(roi_spectra):
    df_peaks = roi_ds.to_peaks(peaks)
    df_peaks["oil_type"] = f"segment_{idx}"
    dfs.append(df_peaks)
peak_df = pd.concat(dfs, ignore_index=True)

print("Peak table:")
print(peak_df)

# Run RQ
cfg = RQConfig(oil_col="oil_type", matrix_col="matrix", heating_col="heating_stage")
res = RatioQualityEngine(peaks=peaks, ratios=ratios, config=cfg).run_all(peak_df)

print("\nRQ Report (first 15 lines):")
print("\n".join(res.text_report.splitlines()[:15]))

## Key Takeaways

### What We Learned:

1. **3D Cube Structure:** HSI data has spatial (x, y) and spectral (wavenumber) dimensions
2. **Spatial Segmentation:** K-means finds meaningful regions based on spectral similarity
3. **ROI Aggregation:** Averaging pixels in a region improves signal-to-noise
4. **Spatial RQ:** Quality metrics now apply to spatial regions, not individual samples
5. **Mapping:** Visual labels show where good/bad regions are located

### Real-World Use:

- Detect bruises/defects in produce
- Find contaminated regions on surfaces
- Map maturity or quality across fruit
- Create quality scores as spatial heatmaps

### Next Steps:
,
,
,
,

: {
: {
: 
3
,
: 
,
: 

: {
: 
,
: 
3.1

: 4,
: 4